In [11]:
import pandas as pd
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import os
import seaborn as sns
import pandas as pd
from math import e,sqrt
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.optimize import leastsq
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import datetime
import statistics
import math
import os
import sys
import statistics
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from embfts.util.DataSetUtil import DataSetUtil
from embfts.util.StatisticsUtil import StatisticsUtil

from sklearn.model_selection import train_test_split

In [13]:
data_set_util = DataSetUtil()
statistics_util = StatisticsUtil()

In [14]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [15]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/energydata_complete.csv', sep=',')
data = df.drop(labels=['date','rv1','rv2'], axis=1)
data = data_set_util.clean_dataset(data)
data = data_set_util.series_to_supervised_mimo(data, 1, 1)
data.head()

,Appliances(t-1),lights(t-1),T1(t-1),RH_1(t-1),T2(t-1),RH_2(t-1),T3(t-1),RH_3(t-1),T4(t-1),RH_4(t-1),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
1,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9
5,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.29,6.016667,734.0,92.0,5.333333,43.833333,4.8


In [16]:
data_red = data.loc[:10000,:]
x,y=data_red.loc[:,:],data_red.loc[:,:]

In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
df_testx=x_test.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
df_trainy=y_train.loc[:,'Appliances(t)':'Tdewpoint(t)']
df_original=y_test.loc[:,'Appliances(t)':'Tdewpoint(t)']

n_inputs, n_outputs = df_trainx.shape[1], df_trainy.shape[1]

In [18]:
print(n_outputs,n_inputs)

26 26


## Randomized_Grid_SearcgCV

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV


model = MultiOutputRegressor(GradientBoostingRegressor())

hyperparameters = dict(estimator__learning_rate=[0.05, 0.1, 0.2, 0.5, 0.9],
                     estimator__n_estimators=[20, 50, 100, 200, 300],
                     estimator__criterion=['friedman_mse', 'mse'], estimator__min_samples_split=[2,3,5,7,9],
                     estimator__max_depth=[2,5,10,20,30], estimator__min_samples_leaf=[1,2,3,4,5,7,9])

randomized_search = RandomizedSearchCV(model, hyperparameters, random_state=0, n_iter=5, scoring=None,
                                       n_jobs=2, refit=True, cv=5, verbose=True,
                                       pre_dispatch='2*n_jobs', error_score='raise', return_train_score=True)

hyperparameters_tuning = randomized_search.fit(df_trainx, df_trainy)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_



Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [ ]:
y_p=tuned_model.predict(df_testx)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

In [ ]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)

In [ ]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca